INFO-H-515 Project <br>
2022–2023

# Phase 1 : Sliding Window Processing
Dimitris Sacharidis, Antonios Kontaxakis <br>
EPB, ULB 

## Information
Group Number : 5 <br>
Group Members : Rania Baguia (000459242), Hakim Amri (000459153), Julian Cailliau (000459856), Mehdi Jdaoudi (000457507)

## Imports

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, col, rank, monotonically_increasing_id
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType
from pyspark.streaming import StreamingContext
import socket
import pandas as pd
import pickle
import json
import numpy as np
import os
import logging
import re, ast
import time
import math

## Configuring the consumer

In [2]:
spark = SparkSession \
    .builder \
    .master("local[2]")\
    .config("spark.executor.instances", "1") \
    .config("spark.executor.cores", "2") \
    .config("spark.executor.memory", "16G") \
    .appName("Windowing") \
    .getOrCreate()

# Let us retrieve the sparkContext object
sc=spark.sparkContext

sc.setLogLevel("ERROR")
logger = spark._jvm.org.apache.log4j
logging.getLogger("py4j.java_gateway").setLevel(logging.ERROR)

23/05/14 19:06:03 WARN Utils: Your hostname, Mehdis-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.205 instead (on interface en0)
23/05/14 19:06:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/14 19:06:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/14 19:06:04 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


#### Set up the Stream function
This function will create a streaming context from a network socket.

In [3]:
def getDStream(sc, batch_interval):
    """
    Create a streaming context and a DStream from a network socket.

    Args:
        sc (SparkContext): The Spark context object.
        batch_interval (int): The time interval in seconds at which streaming data will be divided into batches.

    Returns:
        list: A list containing the streaming context (ssc) and the DStream (dstream).

    Raises:
        None

    Example:
        >>> sc = SparkContext(appName="StreamingExample")
        >>> ssc, dstream = getDStream(sc, 5)
    """

    #Create streaming context, with required batch interval
    ssc = StreamingContext(sc, batch_interval)

    #Checkpointing needed for stateful transforms
    ssc.checkpoint("checkpoint")
    
    # Create a DStream that represents streaming data from a network socket
    dstream = ssc.socketTextStream("localhost", 9999)
    
    return [ssc,dstream]

In this, as we are computing the correlation during the window, there is no states needed. The formula for the pearson coefficient is the same as the stream processing question.
\begin{equation*}
r_{xy} = \dfrac{n\sum x_i y_i - \sum x_i \sum y_i }{\sqrt{n\sum x_i^2-(\sum x_i)^2}\sqrt{n\sum y_i^2-(\sum y_i)^2}}
\end{equation*}

## Runnning the stream
The following cells will run the top-5 computations.

In [4]:
WINDOWS_LENGTH = 40
SLIDE = 10
BATCH_INTERVAL = 10
N_SENSORS = 18

In [5]:
# Get the DStream object containing the streaming data sent by the producer notebook
[ssc,dstream]=getDStream(sc, BATCH_INTERVAL)

dataInWindow = dstream\
    .flatMap(lambda x: [*np.array(ast.literal_eval(x))])\
    .map(lambda x: (x[0], int(x[1]), int(x[2]), int(x[3]), x[4], int(x[5])))\
    .window(WINDOWS_LENGTH, SLIDE)

# Group the data by sensor
dataInWindowPerSensor = dataInWindow\
    .map(lambda x: (x[4], x))\
    .groupByKey()

# Computing the cumulative sum
cumSums = dataInWindowPerSensor\
    .mapValues(lambda x : sum([i[2] for i in x]))\
    .map(lambda x :(x[0], x[1]))\

# Computing the squared cumulative sum
cumSumsSquared = dataInWindowPerSensor\
    .mapValues(lambda x : sum([i[2]**2 for i in x]))\
    .map(lambda x :(x[0], x[1]))\

def compute_covariance_at_T(x, indexes):
    """
    Computes the covariance at time T for a given set of indexes.

    Args:
        x: The input data.
        indexes: The indexes indicating which covariances to compute.

    Returns:
        The computed covariances at time T.
    """
    x = sorted(x, key = lambda y : y[4])
    covAtT = [[(row[4] + "-" + col[4], row[2]*col[2]) for col in x] for row in x]
    covAtT = [covAtT[i[0]][i[1]] for i in indexes]
    return covAtT

# Computing the indexes for the covariance matrix
# Grouping the records per timestamp and computing the covariance matrix at time T
indexes = [(i, j) for i in range(N_SENSORS) for j in range(N_SENSORS) if j > i]
Cum_Covariances = dataInWindow\
    .map(lambda x: (x[5], x))\
    .groupByKey()\
    .mapValues(lambda x : compute_covariance_at_T(x, indexes))\
    .flatMap(lambda x : [record for record in x[1]])\
    .groupByKey()\
    .mapValues(lambda x : sum(x))

# Obtaining the number of observations 
N = dataInWindow\
    .map(lambda x:1)\
    .reduce(lambda x,y:x+y)\
    .map(lambda x : x/N_SENSORS)

def get_correlation(x):
    """
    Computes the correlation using the provided formula.

    Args:
        x: The input data.

    Returns:
        Tuple: A tuple containing the pair identifier and the computed correlation.
    """
    try :
        corr = (x[7]*x[2]-x[3]*x[5])/((math.sqrt(x[7]*x[4]-(x[3]**2)))*(math.sqrt(x[7]*x[6]-(x[5]**2))))
    except ZeroDivisionError:
        corr = -np.inf
    return (x[0] + "-" + x[1], corr)

# Joining the streams to compute the correlation
Correlation = Cum_Covariances\
    .map(lambda x : (*x[0].split("-"), x[1]))\
    .map(lambda x : (x[0], x))\
    .join(cumSums)\
    .map(lambda x : (x[0], (*x[1][0], x[1][1])))\
    .join(cumSumsSquared)\
    .map(lambda x : (x[0], (*x[1][0], x[1][1])))\
    .map(lambda x : (x[1][1], x[1]))\
    .join(cumSums)\
    .map(lambda x : (x[0], (*x[1][0], x[1][1])))\
    .join(cumSumsSquared)\
    .map(lambda x : (*x[1][0], x[1][1]))\
    .transformWith(lambda rdd1, rdd2: rdd1.cartesian(rdd2), N)\
    .map(lambda x : (*x[0], x[1]))\
    .map(get_correlation)

# Filtering to get the top 5 correlations
Top_5_per_Window = Correlation\
    .transform(lambda rdd:rdd.ctx.parallelize(rdd.takeOrdered(5, lambda x: -x[1])))

#Top_5_per_Window.pprint()
Top_5_per_Window.pprint()

/opt/anaconda3/lib/python3.9/site-packages/pyspark/streaming/context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(


In [6]:
ssc.start()

-------------------------------------------
Time: 2023-05-14 19:06:20
-------------------------------------------
('CB02411-CEK049', 0.7657636106436597)
('CEK049-CJM90', 0.7500074623328697)
('CB02411-CJM90', 0.7341503240487527)
('CB2105-CJM90', 0.7071314753542198)
('CB2105-CEK049', 0.6943233481656088)



-------------------------------------------
Time: 2023-05-14 19:06:30
-------------------------------------------
('CEK049-CJM90', 0.8895591661284046)
('CB02411-CEK049', 0.8806025427740716)
('CB02411-CJM90', 0.8799457993377243)
('CB1143-CB2105', 0.7056776812543091)
('CB1143-CJM90', 0.6933695688222551)



In [7]:
ssc.stop(stopSparkContext=False,stopGraceFully=False)

23/05/14 19:06:44 ERROR ReceiverTracker: Deregistered receiver for stream 0: Stopped by driver
Exception in thread "receiver-supervisor-future-0" java.lang.InterruptedException: sleep interrupted
	at java.base/java.lang.Thread.sleep(Native Method)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.$anonfun$restartReceiver$1(ReceiverSupervisor.scala:196)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:659)
	at scala.util.Success.$anonfun$map$1(Try.scala:255)
	at scala.util.Success.map(Try.scala:213)
	at scala.concurrent.Future.$anonfun$map$1(Future.scala:292)
	at scala.concurrent.impl.Promise.liftedTree1$1(Promise.scala:33)
	at scala.concurrent.impl.Promise.$anonfun$transform$1(Promise.scala:33)
	at scala.concurrent.impl.CallbackRunnable.run(Promise.scala:64)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPo

-------------------------------------------
Time: 2023-05-14 19:06:40
-------------------------------------------
('CEK049-CJM90', 0.819021704421524)
('CB02411-CJM90', 0.7778404487961231)
('CB02411-CEK049', 0.7759358889102512)
('CB1143-CEK049', 0.7043333216786912)
('CB1599-CLW239', 0.6986806136245722)



### Stopping the spark session

In [8]:
sc.stop()